<a href="https://colab.research.google.com/github/otanet/AIX_Seminar_GPT-2_2021_Day1_2/blob/main/AIX_Seminar_2021_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**メニュー「ランタイム→ランタイムのタイプを変更」**でハードウェアアクセラレータを**GPU**に変更して保存してください．

In [ ]:
# GoogleDriveをマウントする
from google.colab import drive 
drive.mount('/content/drive')

#作業用フォルダの作成
!mkdir -p '/content/drive/My Drive/AIX_seminner_2021/'

#青空文庫データ保存用フォルダの作成
!mkdir -p '/content/drive/My Drive/AIX_seminner_2021/aozora_data'

# 学習済みモデル保存用フォルダの作成
!mkdir -p '/content/drive/My Drive/AIX_seminner_2021/bert_data'

#作業用フォルダに移動する
%cd '/content/drive/My Drive/AIX_seminner_2021/'

Mounted at /content/drive
/content/drive/My Drive/AIX_seminner_2021


**マイドライブ＞AIX_seminner_2021>aozora_data** 内に青空文庫からダウンロードしたzipファイルを入れてください．
[青空文庫](https://www.aozora.gr.jp/index.html)

In [ ]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

# transformersのインストール
!pip install transformers==4.4.2 tqdm

     |████████████████████████████████| 159 kB 5.3 MB/s 
     |████████████████████████████████| 69 kB 6.8 MB/s 
     |████████████████████████████████| 243 kB 44.2 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0
     |████████████████████████████████| 1.1 MB 5.4 MB/s 
     |████████████████████████████████| 2.0 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 3.3 MB 27.6 MB/s 


In [ ]:
#google drive用ダウンロードツールのインストール
!pip install gdown

In [ ]:
# 作業フォルダに戻る
%cd '/content/drive/My Drive/AIX_seminner_2021/'

[Errno 2] No such file or directory: '/content/drive/My Drive/AIX_seminner_2021/'
/content


In [ ]:
#aozora_dataファルダに移動
%cd '/content/drive/My Drive/AIX_seminner_2021/aozora_data'
#フォルダ内の既存のtxtファイルをすべて削除
!rm *.txt
#フォルダ内のzipファイルを展開する
!unzip '*.zip'
#作業フォルダに移動
%cd '/content/drive/My Drive/AIX_seminner_2021/'

/content/drive/My Drive/AIX_seminner_2021/aozora_data
Archive:  773_ruby_5968.zip
Made with MacWinZipper™
  inflating: kokoro.txt              

Archive:  875_ruby_3249.zip
Made with MacWinZipper™
  inflating: chikyu_hakkyo_jiken.txt  

Archive:  1465_ruby_16804.zip
Made with MacWinZipper™
  inflating: kanikosen.txt           

Archive:  45621_ruby_20106.zip
  inflating: ango_suji.txt           

Archive:  2714_ruby_5772.zip
Made with MacWinZipper™
  inflating: angono_yakuwari.txt     

Archive:  3238_ruby_9508.zip
  inflating: ango_record_jiken.txt   

Archive:  3527_ruby_18323.zip
  inflating: ukabu_hikoto.txt        

Archive:  3355_ruby_13869.zip
  inflating: uchu_senpei.txt         

Archive:  3354_ruby_12227.zip
  inflating: uchuno_maigo.txt        

Archive:  3356_ruby_13870.zip
  inflating: eihondo_joriku.txt      

Archive:  1237_ruby_7222.zip
Made with MacWinZipper™
  inflating: osorosiki_tsuya.txt     

Archive:  1224_ruby_28092.zip
  inflating: kaidan.txt              

Arc

In [ ]:
import re
import glob

#ファインチューニング用のデータを作成する
train_text_list = []
files = glob.glob('./aozora_data/*.txt')
for file in files:
  with open(file, encoding="shift-jis") as f:
    #最終的にtextは句点区切りの文を要素としてもつリストになる
    text = f.read()
    text = re.split('-{55}',text)
    text = re.split('底本：',text[2])
    text = re.sub('《.*》','',text[0])
    text = re.sub('［＃.*］','', text)
    text = re.split("(?<=。)",text)
  #テキストを一文ずつ分割したデータを作成
  for sentence in text[0:-1]:
    if len(sentence):
      train_text_list.append(sentence.strip().replace('\n',''))

#データをtrain.txtとして保存
with open("train.txt", mode='w') as f:
  f.write('\n'.join(train_text_list))
print("Write", len(train_text_list), "lines.")

Write 1901 lines.


In [ ]:
# GPT-2ファインチューニング実行用ファイルのダウンロード
!gdown https://drive.google.com/uc?id=1I9PmXXzLkqaILJLvlq3g6wtVfi8_Z2a1

Downloading...
From: https://drive.google.com/uc?id=1I9PmXXzLkqaILJLvlq3g6wtVfi8_Z2a1
To: /content/drive/My Drive/AIX_seminner_2021/run_clm.py
100% 18.7k/18.7k [00:00<00:00, 5.65MB/s]


In [ ]:
%%time

!rm -r ./output
# ファインチューニングの実行
!python ./run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=3 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output/ \
    --use_fast_tokenizer=False \
    --block_size 512

rm: cannot remove './output': No such file or directory
08/30/2021 15:29:49 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/30/2021 15:29:49 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=output/, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Aug30_15-29-49_27c155066f18, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, logging_steps=500, save_strategy=IntervalStrategy.STEPS, save_steps=5000, save_total_limit=3

**GPT2のみを用いた生成**

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM
import re

# ファインチューニングしたモデルを用いる
USE_FINETUNED_GPT2 = True
# ファインチューニングしたモデルを用いない
# USE_FINETUNED_GPT2 = False

#候補文をいくつ表示するか
OPTION_NUM = 4

# トークナイザーの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
#モデルの準備
if USE_FINETUNED_GPT2:
  model = AutoModelForCausalLM.from_pretrained("output/")
else:
  model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

u = ""
next_sentence = input("\n>")
log = []
log.append(next_sentence)

while(True):  
  if "exit" == u:
    break
  if "back" == u:
    log.pop()
    next_sentence = re.split("(?<=。)",log[-1])[0]
    print("入力文：")
    print(next_sentence)
  if "retry" == u:
    next_sentence = log.pop()
    print("入力文：")
    print(next_sentence)
  if "log" == u:
    print("ログ：")
    print("\n")
    print("\n".join(log))
    print("\n")
    print("入力文：")
    print(next_sentence)
  if not "。" in next_sentence:
    next_sentence = next_sentence + "。" 
  # 推論
  encoded = tokenizer.encode(next_sentence, return_tensors="pt")
  output = model.generate(encoded, do_sample=True, max_length=100, num_return_sequences=OPTION_NUM)

  sequence_list = []
  for sequence in tokenizer.batch_decode(output):
    sequence = sequence.replace('</s>', '')
    sentence_list = re.split("(?<=。)",sequence)[:-1]
    sequence = "".join(sentence_list)
    sequence_list.append(sequence)
  for i,sequence in enumerate(sequence_list):
    print("[", i,"]",sequence)
  
  u = input("\n>")
  if u.isdecimal():
    choice_sequence = sequence_list[int(u)]
    log.append(choice_sequence[len(next_sentence):])
    next_sentence = re.split("(?<=。)", choice_sequence)[-2]
    print("入力文：")
    print(next_sentence)

  # 1. ">"の右の入力欄に最初の一文を入力します
  # 2. 入力文に続く[0]～[3]までの候補文が表示されます
  # 3. 表示された候補の左の数字を">"の右の入力欄に入力することでその候補の最後の文が次の入力文になります
  # 4. "log"と入力するとこれまでの文章が続けて表示されます
  # 5. "back"と入力すると入力が一つ前まで戻ります
  # 6. "retry"と入力すると同じ文で再度文を生成します
  # 7. "exit"と入力すると終了します

KeyboardInterrupt: ignored



---
**ここからBERTの学習とそれを活用した推論**


In [ ]:
#ライブラリのインストール
!apt install git make curl xz-utils file
!apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3==0.996.5
!pip install fugashi
!pip install ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
xz-utils is already the newest version (5.2.2-1.3).
xz-utils set to manually installed.
curl is already the newest version (7.58.0-2ubuntu3.14).
git is already the newest version (1:2.17.1-1ubuntu0.8).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updat

In [ ]:
import os
import re
import glob
import random

#aozora_dataフォルダ内のtxtファイルをつかってpairデータを作成する
pair_list = []
files = glob.glob('./aozora_data/*.txt')
for file in files:
  with open(file, encoding="shift-jis") as f:
    #最終的にtextは句点区切りの文を要素としてもつリストになる
    text = f.read()
    text = re.split('-{55}',text)
    text = re.split('底本：',text[2])
    text = re.sub('《.*》','',text[0])
    text = re.sub('［＃.*］','', text)
    text = re.split("(?<=。)",text)
    #1～3文と1～3文が対応するペアデータを作成
    for i in range(len(text)-6):
      m = random.randint(1,3)
      n = random.randint(1,3)
      pair_list.append("".join(text[i:i+m])+"\t"+"".join(text[i+m:i+m+n]))

#ペアデータをpair.txtとして保存
with open("pair.txt", mode='w') as f:
  f.write('\n'.join(pair_list))

In [ ]:
#BERTの学習用ファイルのダウンロード
!gdown https://drive.google.com/uc?id=1VOsR57Zoyy97lxDMGF1euTzNQJpyoZ7E

Downloading...
From: https://drive.google.com/uc?id=1VOsR57Zoyy97lxDMGF1euTzNQJpyoZ7E
To: /content/run_glue.py
100% 23.4k/23.4k [00:00<00:00, 46.2MB/s]


In [ ]:
write_lines = []
uttrs = []

filename = 'pair.txt'

with open(filename) as f:
    for l in f:
        l = l.strip()
        if "\t" in l:
            # 実際の応答ペアを正解とし，ラベルは1とする．
            write_lines.append('1,"' + l.replace("\t", "") + '"' + "\n")
            # 不正解ペアの作成のため，発話を保存
            uttrs.append(l.split("\t")[0])
            uttrs.append(l.split("\t")[1])
  
# 正解ペアと同じ数だけ不正解ペアを作成
for i in range(len(write_lines)):
    # ランダムな応答ペアを不正解とし，ラベルは0とする．
    write_lines.append('0,"' + random.choice(uttrs) + random.choice(uttrs) + '"' + "\n")
  
 # 正解ペアと不正解ペアが入ったリストをシャッフルする
random.shuffle(write_lines)
  
index = 0
with open("bert_data/dev.csv", "w") as var_f:
    var_f.write("label,sentence\n")
    # 開発データとしてdev.tsvに200行を書き込む．
    for l in write_lines[:200]:
        var_f.write(l)
        index += 1
index = 0
with open("bert_data/train.csv", "w") as var_f:
    var_f.write("label,sentence\n")
    # 学習データとしてtrain.tsvにのこりを書き込む．
    for l in write_lines[200:]:
        var_f.write(l)
        index += 1
print("Write", len(write_lines[200:]), "lines to train.tsv")

Write 46256 lines to train.tsv


In [ ]:
# max_stepsの値を大きな値に設定することで，より多くのデータで学習できるが，より多くの時間が必要となる．
# 時間がかかるので100に設定しているが，実際は全然足らない．10000以上には設定したい．
!python ./run_glue.py --train_file ./bert_data/train.csv --validation_file ./bert_data/dev.csv  --overwrite_output_dir --overwrite_cache \
--model_name_or_path cl-tohoku/bert-base-japanese-whole-word-masking  --save_steps 100 --max_steps 100 \
--output_dir bert_output/ --do_train --do_eval --per_gpu_train_batch_size 16

usage: run_glue.py [-h] --model_name_or_path MODEL_NAME_OR_PATH
                   [--config_name CONFIG_NAME]
                   [--tokenizer_name TOKENIZER_NAME] [--cache_dir CACHE_DIR]
                   [--no_use_fast_tokenizer]
                   [--use_fast_tokenizer [USE_FAST_TOKENIZER]]
                   [--model_revision MODEL_REVISION]
                   [--use_auth_token [USE_AUTH_TOKEN]] [--task_name TASK_NAME]
                   [--max_seq_length MAX_SEQ_LENGTH]
                   [--overwrite_cache [OVERWRITE_CACHE]]
                   [--no_pad_to_max_length]
                   [--pad_to_max_length [PAD_TO_MAX_LENGTH]]
                   [--max_train_samples MAX_TRAIN_SAMPLES]
                   [--max_val_samples MAX_VAL_SAMPLES]
                   [--max_test_samples MAX_TEST_SAMPLES]
                   [--train_file TRAIN_FILE]
                   [--validation_file VALIDATION_FILE] [--test_file TEST_FILE]
                   --output_dir OUTPUT_DIR
                   

In [ ]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class BertEvaluator:
    def __init__(self):
        # 事前学習済みのトークナイザとモデルをロード
        self.tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', do_lower_case=False)
        self.model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=2)
        
        # Google Colabでファインチューニングしたモデルをロード
        self.model.load_state_dict(torch.load("bert_output/pytorch_model.bin", map_location="cpu"))
        self.model.to(device)

    def evaluate(self, user_input, candidate):
        with torch.no_grad():
            # 発話のペアを特徴ベクトルに変換
            tokenized = self.tokenizer([user_input+candidate], return_tensors="pt")
            input_ids = tokenized["input_ids"].to(device)
            token_type_ids = tokenized["token_type_ids"].to(device)

            # ファインチューニング済みのBERTを用いて特徴ベクトルから2文のスコアを計算
            result = self.model.forward(input_ids, token_type_ids=token_type_ids)
            # softmax関数によりスコアを正規化
            result = F.softmax(result[0], dim=1).cpu().numpy().tolist()

            # 結果を返す．
            return result[0][1]

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM
import re

#文章をいくつ推論するか
RETURN_NUM = 8
#候補文をいくつ表示するか(RETURN_NUMより小さい値にしてください)
OPTION_NUM = 4

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("output/")
#BERTを使った評価用クラスの準備
be = BertEvaluator()

u = ""
next_sentence = input("\n>")
log = []
log.append(next_sentence)

while(True):  
  if "exit" == u:
    break
  if "back" == u:
    log.pop()
    next_sentence = re.split("(?<=。)",log[-1])[0]
    print("入力文：")
    print(next_sentence)
  if "retry" == u:
    next_sentence = log.pop()
    print("入力文：")
    print(next_sentence)
  if "log" == u:
    print("ログ：")
    print("\n")
    print("\n".join(log))
    print("\n")
    print("入力文：")
    print(next_sentence)
  if not "。" in next_sentence:
    next_sentence = next_sentence + "。" 
  # 推論
  encoded = tokenizer.encode(next_sentence, return_tensors="pt")
  output = model.generate(encoded, do_sample=True, max_length=100, num_return_sequences=RETURN_NUM)

  sequence_dict = {}
  for sequence in tokenizer.batch_decode(output):
    sequence = sequence.replace('</s>', '')
    sentence_list = re.split("(?<=。)",sequence)[:-1]
    sequence = "".join(sentence_list)
    score = be.evaluate("".join(log[-1]), sequence[len(next_sentence):])
    sequence_dict[sequence] = score
  sequence_dict = sorted(sequence_dict.items(), key=lambda x:x[1], reverse=True)
  for i in range(OPTION_NUM):
    print("[", i,"]", sequence_dict[i][0])
  
  u = input("\n>")
  if u.isdecimal():
    choice_sequence = sequence_dict[int(u)][0]
    log.append(choice_sequence[len(next_sentence):])
    next_sentence = re.split("(?<=。)", choice_sequence)[-2]
    print("入力文：")
    print(next_sentence)

  # 1. ">"の右の入力欄に最初の一文を入力します
  # 2. 入力文に続く[0]～[3]までの候補文が表示されます
  # 3. 表示された候補の左の数字を">"の右の入力欄に入力することでその候補の最後の文が次の入力文になります
  # 4. "log"と入力するとこれまでの文章が続けて表示されます
  # 5. "back"と入力すると入力が一つ前まで戻ります
  # 6. "retry"と入力すると同じ文で再度文を生成します
  # 7. "exit"と入力すると終了します

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize


>今日はいい天気だ。


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[ 0 ] 今日はいい天気だ。 日が沈むのもそろそろだろう。 「今はもう日の沈むのも早くなったね。 「お前が夕焼けを見たいっていい出すんだから、それで日没が早いって言うんだろう」 そういって彼は私の方を向いて向うを向いてくれた。
[ 1 ] 今日はいい天気だ。 こんな日でいいのかと自分で思ってしまう。 「もうすぐ雨になるんですか」 私は突然そんな風に尋ねた。
[ 2 ] 今日はいい天気だ。 外が少し明るい。 窓から差し込む光は明るいように思われるけれども、外を見る限りそれは全く感じられない。 私は何だか変な風を切望していた。
[ 3 ] 今日はいい天気だ。 今から散歩をしよう。 今日はどこへ行こうかな。 あそこへでも。


KeyboardInterrupt: ignored